In [ ]:
import math
import graphviz
import math
import copy
import sys
import arff

------
### Algorytm TABU
<img src="../img/tabu.png" style="height: 350px">

In [ ]:
class TabuSearch(object):
    def __init__(self, attributes, sample_data, scoring_method, max_number_of_parents, number_of_iterations):
        self.attributes = attributes
        self.sample_data = sample_data
        self.scoring_method = scoring_method
        self.max_number_of_parents = max_number_of_parents
        self.number_of_iterations = number_of_iterations
        self.tabu_length = len(attributes)
        self.graph_checker = GraphChecker()
        
    def compute_optimal_net(self):
        iteration = 0
        network = self.prepare_initial_network()
        tabu_list = []
        best_score = self.compute_metric(network)
        
        while(iteration < self.number_of_iterations):
            iteration += 1
            
            child, parent, best_candidate = self.find_best_addition(network, tabu_list)
            tabu_list.append({"child": child, "parent": parent})
            
            best_candidate_score = self.compute_metric(best_candidate)
            if best_candidate_score > best_score:
                best_score = best_candidate_score
                network = best_candidate
            
            self.check_tabu_list_length(tabu_list)
        
        return network
            
    def find_best_addition(self, network, tabu):
        temp_network = copy.deepcopy(network)
        best_network = copy.deepcopy(network)
        best_parent = ""
        best_child = ""
        best_score = -sys.maxsize -1
        
        #finding best addition: first_node (parent) → second_node (child)
        for first_node in temp_network:
            for second_node in temp_network:
                if (first_node['name'] != second_node['name'] 
                    and not self.check_if_node_already_has_parent(second_node['name'], first_node['name'], best_network) 
                    and not self.has_max_parents(second_node['name'], best_network)
                    and not self.is_in_tabu(second_node['name'], first_node['name'], tabu)):
                    
                    candidate_network = self.add_parent_to_node(second_node['name'], first_node, copy.deepcopy(temp_network))
                    if self.graph_checker.is_graph_acyclic(candidate_network):
                        continue
                    
                    score = self.compute_metric(candidate_network)

                    # print(first_node['name'], "→", second_node['name'], ":", score)
                    
                    if (score > best_score):
                        best_score = score
                        best_parent = first_node['name']
                        best_child = second_node['name']
                        best_network = candidate_network

        return (best_child, best_parent, best_network)
    
    def is_in_tabu(self, child, parent, tabu):
        for entry in tabu:
            if entry['child'] == child and entry['parent'] == parent:
                return True
        return False
    
    def prepare_initial_network(self):
        initial_bayes_network = []
        for attribute in self.attributes:
            initial_bayes_network.append({'r': attribute['states'], 'name': attribute['name'], 'parents': []})
        return initial_bayes_network
    
    def check_tabu_list_length(self, tabu_list):
        if len(tabu_list) == self.tabu_length:
            del tabu_list[0]
            
    def check_if_node_already_has_parent(self, child_name, parent_name, net):
        for node in net:
            if node['name'] == child_name:
                for parent in node['parents']:
                    if parent['name'] == parent_name:
                        return True
        return False

    def has_max_parents(self, node_name, net):
        for node in net:
            if node['name'] == node_name:
                if len(node['parents']) >= self.max_number_of_parents:
                    return True
                else:
                    return False
        raise ValueError(self.node_name + " is not found in given net")
        
    def add_parent_to_node(self, node_name, parent, net):
        for node in net:
            if node['name'] == node_name:
                node['parents'].append({'name': parent['name'], 'q': parent['r']})
        return net
        
    def compute_metric(self, net):
        if self.scoring_method == 'aic':
            return AICMetric().compute_aic_metric(net, self.sample_data)
        elif self.scoring_method == 'mdl':
            return MDLMetric().compute_mdl_metric(net, self.sample_data)
        elif self.scoring_method == 'bayes':
            return BayesianMetric(net, self.sample_data).compute_bayesian_metric()
        
        raise ValueError(self.scoring_method + " is not a valid scoring method!")
        
    def prepare_graph(self, net):
        graph = []
        
        for node in net:
            graph.append({"node": node['name'], "children": []})
            
        for node in net:
            for parent in node['parents']:
                for graph_node in graph:
                    if graph_node['node'] == parent['name']:
                        graph_node['children'].append(node['name'])
                        
        return graph